## Data Cleaning
----
In this interactive notebook, we explore the process of cleaning up daily summary data downloaded en masse through the AQS website. The website provides ZIP archives of CSV files for each of the parameters of Criteria gasses, which are all contained in the AQI defined pollutants class of parameters. I have collected ZIP files with daily summary data for each of Carbon Monoxide (CO), Ozone (O3), Sulfur Dioxide (SO2), and Nitrogen Dioxide (NO2) from 2010 to 2018.

Let's start by importing our required libraries.

In [47]:
import pandas as pd
from pyaqs import AQSFetcher

Next, let's instantiate an object that will handle fetching some parameter and state codes from the AQS API.

In [48]:
aqs_fetcher = AQSFetcher('bbjornstad.flatiron@gmail.com', 'ochrefox21')

And now, we can use this object to get our needed codes. We will also save these into the `cleaned_data` folder so that they can also be easily accessed for annotation and iteration purposes later on.

In [51]:
param_codes = aqs_fetcher.get_parameter_list_by_class('AQI POLLUTANTS')
aqi_gasses = param_codes.iloc[:4,:]
aqi_gasses['param_shorthand'] = ['CO', 'SO2', 'NO2', 'Ozone']
aqi_gasses['legend_entry'] = aqi_gasses.apply(lambda r: f'{r.code} - {r.param_shorthand}', axis=1)

state_codes = aqs_fetcher.get_state_codes()
# filter out some junk locations that don't have full data
state_codes = state_codes.loc[state_codes.code != 'CC']
state_codes.code = state_codes.code.astype(int)
state_codes = state_codes.loc[state_codes.code < 60]

In [ ]:
aqi_gasses.to_csv('cleaned_data/aqi_gasses.csv', index=False)
state_codes.to_csv('cleaned_data/state_codes.csv', index=False)

Next, let's create a variable to hold the root location of all of our air quality data sources for our selected gasses. These files are stored in ZIP format.

In [52]:
data_folder = 'raw_data/'

Let's store a list holding all of the years for which we have downloaded daily summary data for each of the above four gas parameters.

In [54]:
date_range = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

Now, with this, we can import and clean up our datasets. Let's create a master list of unneeded columns, so that we can easily drop them from each imported dataset.

In [55]:
cols_to_drop = [
    'POC',
    'Latitude',
    'Longitude',
    'Datum',
    'Pollutant Standard',
    'Event Type',
    'Observation Count',
    'Observation Percent',
    '1st Max Value',
    '1st Max Hour',
    'Method Code',
    'Method Name',
    'Address',
    'CBSA Name',
    'Date of Last Change',
    'Local Site Name']

To ensure that our data is handled in the same fashion for each file, we should probably write a function that can handle importation of a particular parameter code for a particular year. This corresponds to one particular file within the `raw_data` directory.

In [56]:
def import_aggregate_data(param_code, year):
    path_stub = f'/daily_{param_code}_{year}.zip'
    path = data_folder+path_stub
    df = pd.read_csv(path)
    df.drop(columns = cols_to_drop, inplace=True)
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.date_local = pd.to_datetime(df.date_local)
    return df

With this we can import our dataframes for each gas to create a master dataframe, on which we will perform some aggregation to make our data formatted appropriately for our future modeling.

In [57]:
all_data_raw_agg_trimmed = pd.DataFrame()

for param_code in aqi_gasses.code:
    all_param_df = pd.DataFrame()
    for year in date_range:
        yearly_param_df = import_aggregate_data(param_code, year)
        all_param_df = pd.concat([all_param_df, yearly_param_df])
        
    all_param_df.to_csv(f'intermediate_data/{param_code}_daily.csv')
    
    all_data_raw_agg_trimmed = pd.concat([all_data_raw_agg_trimmed, all_param_df])
    print(f'Imported {param_code}')

42101
42401
42602


//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


44201


After importation, we should check some basic descriptive information provided by Pandas to ensure that everything has been appropriately imported and is ready for formatting and further filtering.

In [58]:
all_data_raw_agg_trimmed.shape

(9710154, 13)

In [59]:
all_data_raw_agg_trimmed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9710154 entries, 0 to 402633
Data columns (total 13 columns):
state_code          int64
county_code         int64
site_num            int64
parameter_code      int64
parameter_name      object
sample_duration     object
date_local          datetime64[ns]
units_of_measure    object
arithmetic_mean     float64
aqi                 float64
state_name          object
county_name         object
city_name           object
dtypes: datetime64[ns](1), float64(2), int64(4), object(6)
memory usage: 1.0+ GB


We are most interested in the `aqi` column, which represents the calculated Air Quality Index per the EPA standards of calculation, which is well defined for each of the four selected gasses. However, not all of the AQI entries are populated appropriately. Because of the massive collection of data that we have managed to access, we can safely simply drop these entries from our dataframe to ensure that we have all non-null data for future modeling.

In [60]:
all_data_no_nan_aqi = all_data_raw_agg_trimmed.dropna(subset=['aqi'])

In [61]:
all_data_no_nan_aqi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7262623 entries, 355 to 402633
Data columns (total 13 columns):
state_code          int64
county_code         int64
site_num            int64
parameter_code      int64
parameter_name      object
sample_duration     object
date_local          datetime64[ns]
units_of_measure    object
arithmetic_mean     float64
aqi                 float64
state_name          object
county_name         object
city_name           object
dtypes: datetime64[ns](1), float64(2), int64(4), object(6)
memory usage: 775.7+ MB


In [63]:
all_data_no_nan_aqi.date_local.value_counts()

2016-05-26    2609
2016-07-22    2584
2016-07-20    2584
2016-05-24    2577
2016-05-23    2548
              ... 
2010-11-28    1738
2010-12-24    1737
2010-12-27    1734
2010-12-25    1734
2010-12-26    1729
Name: date_local, Length: 3287, dtype: int64

In [64]:
all_data_no_nan_aqi.sample_duration.value_counts()

8-HR RUN AVG BEGIN HOUR    3502101
1 HOUR                     2793432
8-HR RUN AVG END HOUR       967090
Name: sample_duration, dtype: int64

In [69]:
all_data_no_nan_aqi = all_data_no_nan_aqi.loc[all_data_no_nan_aqi.state_name.isin(state_codes.state_name)]
len(all_data_no_nan_aqi.state_name.unique())

51

Now that we have appropriately filtered the data, and ensured that all entries for the Air Quality Index are non-null, we aim to take an average AQI reading for each state on each day for each of the specified four parameters. This is easily accomplished with a Pandas groupby statement. After grouping, we will have a few superfluous columns whose meaning will have been lost, and we can safely remove these from our dataset.

In [70]:
mean_aggregation_no_nan = all_data_no_nan_aqi.groupby(['state_name', 'parameter_code', 'date_local']).mean()
mean_aggregation_no_nan = mean_aggregation_no_nan.drop(columns=['county_code', 'site_num', 'state_code'])

In [71]:
mean_aggregation_no_nan.head(10)

arithmetic_mean        aqi
state_name parameter_code date_local                            
Alabama    42101          2010-01-01         0.280702   3.333333
                          2010-01-02         0.288889   3.333333
                          2010-01-03         0.277778   3.333333
                          2010-01-04         0.361111   5.333333
                          2010-01-05         0.404166   5.333333
                          2010-01-06         0.413889  10.666667
                          2010-01-07         0.719444  14.333333
                          2010-01-08         0.306944   4.333333
                          2010-01-09         0.283333   4.000000
                          2010-01-10         0.329167   6.666667

Finally, we can save our cleaned and aggregated data to our `cleaned_data` folder, so that we can easily access the dataset in future modeling and analysis.

In [72]:
mean_aggregation_no_nan.to_csv('cleaned_data/criteria_gasses_aqi_means.csv')